# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [5]:
import pandas as pd

In [9]:
df = pd.read_csv('SMSSpamCollection', sep="\t", header=None)
df.columns = ['type','text']
df.head()

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [10]:
all_ham = df[df.type == 'ham']
all_spam = df[df.type == 'spam']
print("Total # of ham:", len(all_ham))
print("Total # of spam:", len(all_spam))

Total # of ham: 4825
Total # of spam: 747


In [11]:
ham_subset = all_ham.sample(n=747, random_state=123)
len(ham_subset)

747

## Train-test split

Now implement a train-test split on the dataset: 

In [12]:
# Your code here
from sklearn.model_selection import train_test_split
data = pd.concat([ham_subset, all_spam], axis=0)
X = data.text
y = data.type
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)
train_df = pd.concat([X_train, y_train], axis = 1)
test_df = pd.concat([X_test, y_test], axis = 1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [14]:
class_dicts = {}
for class_ in train_df.type.unique():
    temp = train_df[train_df.type == class_]
    word_freqs = {}
    for row in temp.index:
        for word in temp['text'][row].split():
            word_freqs[word] = word_freqs.get(word, 0) + 1
    class_dicts[class_] = word_freqs

In [15]:
class_dicts

{'ham': {'Machan': 1,
  'you': 165,
  'go': 28,
  'to': 202,
  'gym': 1,
  'tomorrow,': 3,
  'i': 68,
  'wil': 2,
  'come': 21,
  'late': 2,
  'goodnight.': 1,
  'We': 10,
  'can': 39,
  '4': 19,
  'e': 6,
  'normal': 1,
  'pilates': 1,
  'after': 7,
  'our': 4,
  'intro...': 1,
  'I': 145,
  'don': 2,
  'know': 19,
  'account': 3,
  'details..i': 1,
  'will': 34,
  'ask': 5,
  'my': 72,
  'mom': 3,
  'and': 93,
  'send': 11,
  'you.my': 1,
  'is': 69,
  'out': 23,
  'of': 57,
  'reach': 4,
  'now.': 14,
  'Yeah': 3,
  'work': 8,
  'fine,': 1,
  'started': 2,
  'last': 10,
  'week,': 1,
  'all': 26,
  'the': 123,
  'same': 2,
  'stuff': 3,
  'as': 15,
  'before,': 1,
  'dull': 1,
  'but': 26,
  'easy': 2,
  'guys': 2,
  'are': 35,
  'fun!': 1,
  "Don't": 2,
  'forget': 3,
  'though': 3,
  'that': 49,
  'love': 13,
  '....': 2,
  'And': 11,
  'walk': 2,
  'beside': 1,
  'you.': 13,
  'Watching': 4,
  'over': 8,
  'keeping': 2,
  'your': 49,
  'heart': 2,
  'warm.': 1,
  'Also': 4,
  "wh

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [16]:
vocab = set()
for text in train_df.text:
    for word in text.split():
        vocab.add(word)
        
V = len(vocab)
print(V)

6113


## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [17]:
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [18]:
import numpy as np

In [19]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    class_ps = {}
    bag = bag_it(doc)
    for class_ in p_classes.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word] + 1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num / denom)
        class_ps[class_] = p
    
    return max(class_ps, key = lambda key: class_ps[key])

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [21]:
p_classes = dict(train_df.type.value_counts(normalize=True))
p_classes

{'ham': 0.5080357142857143, 'spam': 0.4919642857142857}

In [23]:
y_hat_train = [classify_doc(train_df['text'][row], class_dicts, p_classes, V) for row in train_df.index]
y_hat_test = [classify_doc(test_df['text'][row], class_dicts, p_classes, V) for row in test_df.index]
train_successes = y_hat_train == train_df.type
test_successes = y_hat_test == test_df.type
train_acc = train_successes.sum()/len(train_successes)
test_acc = test_successes.sum()/len(test_successes)

print("Classifier Accuracy for Train Set: {}, Test Set: {}".format(train_acc, test_acc))

Classifier Accuracy for Train Set: 0.49642857142857144, Test Set: 0.47058823529411764


## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!